In [1]:
import numpy as np
import scipy
import pandas as pd

from astropy.table import Table
from astropy import units as u
from astropy import constants as const
from astroquery.gaia import Gaia
from astropy import table

from tqdm import tqdm

import stam

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import ticker
%matplotlib tk

plt.style.use('Nature')

from IPython.display import Image

In [2]:
# Nature standards: https://www.nature.com/nature/for-authors/final-submission
figwidth_single = 89/25.4  # [inch]
figwidth_double = 183/25.4  # [inch]
gaia = Table.read('../data/other/GaiaDR3_100pc_clean.fits')
parsec_imported = False

## Plot CMD

In [3]:
sources = Table.read('../table_C_Dias.fits')
# clusters = Table.from_pandas(pd.read_csv('../data/hunt_clusters/clusters.csv'))
# members = Table.from_pandas(pd.read_csv('../data/hunt_clusters/members.csv'))

In [13]:
PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
# PARSEC_path_00 = '../data/PARSECv2.0/w_i=0.0/'
# PARSEC_path_03 = '../data/PARSECv2.0/w_i=0.3/'
# PARSEC_path_06 = '../data/PARSECv2.0/w_i=0.6/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
# models0 = stam.getmodels.read_parsec(path=PARSEC_path_00)
# models3 = stam.getmodels.read_parsec(path=PARSEC_path_03)
# models6 = stam.getmodels.read_parsec(path=PARSEC_path_06)
parsec_imported = True

In [9]:
def plot_cmd(tables,mrkr_colors,markers,labels,title,color_column='bp_rp',magnitude_column='mg',inclination = [None],plot_isochrone = False, age = [100], mh = [0],omega=[0.6],a_v = 0,alpha=1):
    # get tables in a list
    # colors and markers as list of strings corresponding to each table
    fig, ax = plt.subplots(dpi=300)   
    ax.hist2d(gaia["bp_rp"], gaia["mg"], bins=500, cmap="Greys", norm=colors.PowerNorm(0.5), zorder=0.5) ## background CMD reference

    if not(parsec_imported):
        print('load parsec isochrones')
    elif plot_isochrone:
        stage_min = 0  # pre-main sequence
        stage_max = 10  # red giant branch
        mass_min = 0  # [Msun]
        mass_max = 10  # [Msun]
        
        for a,m,o,i in zip(age,mh,omega,inclination):
            if a <= 45:
                age_res = 5e-4 # [Gyr]
            elif a <= 100:
                age_res = 1e-3
            else:
                age_res = 1e-2
            if i is None:
                color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G"
                l = np.log10(a*1e6)
                log_age_res = 0.025  # [dex]
                model = models
                tracks = stam.gentracks.get_isotrack(model, [l, m], params=("log_age", "mh"), return_table=True,
                                        log_age_res=log_age_res, mass_min=mass_min, mass_max=mass_max,
                                        stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                    mag_filter=mag_fil)
            else:
                if i < 10:
                    inc_suffix = '0' + f'{int(i)}'
                else:
                    inc_suffix = f'{int(i)}'
                if o == 0:
                    model = models0
                elif o == 0.3:
                    model = models3
                elif o == 0.6:
                    model = models6
                color_fil_1, color_fil_2, mag_fil = "G_BP_i" + inc_suffix, "G_RP_i" + inc_suffix, "G_i" + inc_suffix
                tracks = stam.gentracks.get_isotrack(model, [a* 1e-3, m], params=("age", "mh"), return_table=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.02,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)
            x = np.array(tracks["bp_rp"])
            y = np.array(tracks["mg"])
            z = np.array(tracks["mass"])
            ax.plot(x, y, 'o', markersize=0.5,label=f'{float(f"{int(a)}"):g} Myr, M/H = {m:.2f}')

    for tbl,clr,mrkr,lbl,av,al in zip(tables,mrkr_colors,markers,labels,a_v,alpha):
        bp_rp = tbl[color_column] ## before dereddening
        mg = tbl[magnitude_column] ## before dereddening
        # ax.scatter(bp_rp, mg, s=5, c='r') 
        e_bv = av / 3.1
        e_bprp, A_G = stam.gaia.get_extinction_in_band(e_bv,mag_filter="G",color_filter1='G_BP',color_filter2='G_RP')
        bp_rp_corrected = bp_rp - e_bprp
        mg_corrected = mg - A_G
        ax.scatter(bp_rp_corrected, mg_corrected, s=5, marker = mrkr, c=clr,label=lbl,alpha=al)
    ax.invert_yaxis()
    ax.set_xlabel(r"$G_\text{BP} - G_\text{RP}$")
    ax.set_ylabel(r"$G_\text{abs}$")
    ax.set_xlim([-1, 5.5])
    ax.set_ylim([17.6, -2])
    ax.legend(loc='lower left',frameon=False,fontsize=8)
    ax.set_title(title)
    return fig

def plot_isochrones(age, mh, omega, inclination, title):
    fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)   
    ax.hist2d(gaia["bp_rp"], gaia["mg"], bins=500, cmap="Greys", norm=colors.PowerNorm(0.5), zorder=0.5) ## background CMD reference
    for a,m,o,i in zip(age,mh,omega,inclination):
        if a <= 45:
            age_res = 5e-4 # [Gyr]
        elif a <= 100:
            age_res = 1e-3
        else:
            age_res = 1e-2
        stage_min = 0  # pre-main sequence
        stage_max = 10  # red giant branch
        mass_min = 0  # [Msun]
        mass_max = 10  # [Msun]
        if i is None:
            color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G"
            l = np.log10(a*1e6)
            log_age_res = 0.025  # [dex]
            model = models
            tracks = stam.gentracks.get_isotrack(model, [l, m], params=("log_age", "mh"), return_table=True,
                                    log_age_res=log_age_res, mass_min=mass_min, mass_max=mass_max,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)
        else:
            if i < 10:
                inc_suffix = '0' + f'{int(i)}'
            else:
                inc_suffix = f'{int(i)}'
            if o == 0:
                model = models0
            elif o == 0.3:
                model = models3
            elif o == 0.6:
                model = models6
            color_fil_1, color_fil_2, mag_fil = "G_BP_i" + inc_suffix, "G_RP_i" + inc_suffix, "G_i" + inc_suffix
            tracks = stam.gentracks.get_isotrack(model, [a * 1e-3, m], params=("age", "mh"), return_table=True,
                                age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.02,
                                stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
            mag_filter=mag_fil)
        x = np.array(tracks["bp_rp"])
        y = np.array(tracks["mg"])
        z = np.array(tracks["mass"])
        ax.plot(x, y, 'o', markersize=0.5,label=f'{float(f"{int(a)}"):g} Myr, M/H = {m:.2f}, $\Omega = ${o}')
    ax.invert_yaxis()
    ax.set_xlabel(r"$G_\text{BP} - G_\text{RP}$")
    ax.set_ylabel(r"$G_\text{abs}$")
    ax.set_xlim([-1, 5.5])
    ax.set_ylim([17.6, -2])
    ax.legend()
    ax.set_title(title)
    return fig

In [ ]:
## isochrone comparisons

age = [700,700,700]
mh = [0,0,0]
omega = [0,0,0.6]
inclination = [None,45,45]
title = 'Older clusters'

fig = plot_isochrones(age, mh, omega, inclination, title)

In [17]:
## cluster observed CMD + candidate + best isochrone

i_arr = np.arange(len(sources))
for i in i_arr:
    src = sources[i]
    cluster_name = src['cluster']
    nbr = Table.read('../data/clusters2/' + cluster_name + '.fits')
    
    # fld = Table.read(f'../field/field_circ{i+1}.csv')
    
    a_v = src['av_for_mass_interp']
    mh = src['mh_for_mass_interp']
    age = src['age']
    
    nbr['mg'] = nbr['phot_g_mean_mag'] +  5 * np.log10(nbr['parallax']) - 10

    tables = [nbr,src]
    mrkr_colors = ['k','r']
    labels = [f'{len(nbr)} cluster members',f'candidate m1={src["m1"]:.2g}/m2={src["m2"]:.2g}']
    # labels = [f'{len(nbr)} cluster members av = {a_v:1.2f}',f'candidate m1={src["parsec_m1"]:1.1f}/m2={src["parsec_m2"]:1.1f} av = {a_v:1.2f}'
    #                         ,f'members av={a_v2:1.2f}', f'candidate av={a_v2:1.2f}']
    markers = ['o','o']
    alpha = [1,1]
    ttl = f"Cluster " + cluster_name

    # fig = plot_cmd(tables,mrkr_colors,markers,labels,ttl,'bp_rp','mg',True,age,mh,np.array([a_v,a_v]),alpha)
    fig = plot_cmd(tables,mrkr_colors,markers,labels,ttl,'bp_rp','mg',inclination=[None,None],plot_isochrone=True,age=[age,age],mh=[mh,mh],a_v=np.array([a_v,a_v]),alpha=[1,1])
    fig.savefig(f"../img/cmd/cmd"+ cluster_name + ".png",dpi=300)
